In [ ]:
import sys
from pathlib import Path

from extract import convert_pdf_with_vlm

sys.path.append(str(Path().resolve().parent))

In [ ]:
extraction = convert_pdf_with_vlm(
    "/Users/alessandro/Development/generalRAG/data/original/Fact-Sheets.pdf"
)


In [ ]:
extraction

In [ ]:
extraction.document.export_to_dict()

In [29]:
import json

with open("data.json", "w") as f:
    json.dump(extraction.document.export_to_dict(), f)

In [30]:
with open("html_data.json", "w") as f:
    json.dump(extraction.document.export_to_html(), f)

In [31]:
with open("markdown_data.json", "w") as f:
    json.dump(extraction.document.export_to_markdown(), f)

In [33]:
print(len(extraction.document.pictures))

32


In [36]:
from loguru import logger

logger.info("a")

2025-04-08 22:53:47.065 | INFO     | __main__:<module>:2 - a


In [38]:
import sys

print("This goes to notebook output")

# This attempts to write directly to terminal stdout
sys.__stdout__.write("This goes to the terminal (if not captured by Jupyter)\n")

This goes to notebook output
This goes to the terminal (if not captured by Jupyter)


55

In [40]:
log_file = open("log.txt", "a")
log_file.write(f"{len(extraction.document.pictures)}\n")
log_file.flush()  # Ensure it writes immediately


In [1]:
import json
import re

from docling_core.types.doc.document import PictureDescriptionData


def insert_image_annotations(json_path, output_path, doc):
    # Load the original JSON content
    with open(json_path, "r", encoding="utf-8") as f:
        content = json.load(f)

    # Count image tags in the content
    image_tags = list(re.finditer(r"<!--\s*image\s*-->", content))
    image_count = len(image_tags)

    # Ensure the counts match
    if image_count != len(doc.pictures):
        raise ValueError(
            f"Mismatch: {image_count} <!-- image --> tags vs {len(doc.pictures)} images in doc.pictures"
        )

    # Split the content at each image tag
    parts = re.split(r"(<!--\s*image\s*-->)", content)
    print(parts)

    # New buffer for modified content
    new_parts = []
    image_index = 0

    for part in parts:
        new_parts.append(part)
        if part.strip() == "<!-- image -->":
            pic = doc.pictures[image_index]
            html_item = (
                f"<h3>Picture <code>{pic.self_ref}</code></h3>"
                f'<img src="{str(pic.image.uri)}" /><br />'
                f"<h4>Caption</h4>{pic.caption_text(doc=doc)}<br />"
            )
            for annotation in pic.annotations:
                if not isinstance(annotation, PictureDescriptionData):
                    continue
                html_item += f"<h4>Annotations ({annotation.provenance})</h4>{annotation.text}<br />\n"
            new_parts.append(html_item)
            image_index += 1

    # Join and save updated content
    modified_content = "".join(new_parts)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(modified_content, f, ensure_ascii=False, indent=2)

    print(f"Modified content saved to: {output_path}")

In [ ]:
insert_image_annotations(
    json_path="/Users/alessandro/Development/generalRAG/extract/markdown_data.json",
    output_path="markdown_data_with_images.json",
    doc=extraction.document,
)

NameError: name 'extraction' is not defined